In [ ]:
#!/usr/bin/env python
# Perceptron  Example in python without using special libraries

# Dataset : Sonar Dataset - This is a dataset that describes sonar chirp returns bouncing off different services.
# The 60 input variables are the strength of the returns at different angles. 
# It is a binary classification problem that requires a model to differentiate rocks from metal cylinders.
# It is a well-understood dataset. All of the variables are continuous and generally in the range of 0 to 1.
# As such we will not have to normalize the input data, which is often a good practice with the Perceptron algorithm.
# The output variable is a string “M” for mine and “R” for rock, which will need to be converted to integers 1 and 0. 
# You can learn more about this dataset at the UCI Machine Learning repository. 
# You can download the dataset for free and place it in your working directory with the filename sonar.all-data.csv.


# Function predict() predicts an output value for a row given a set of weights. 
# This is needed both in the evaluation of candidate weights values in stochastic gradient descent, 
# and after the model is finalized where we wish to start making predictions on test data or new data.
# Note that the first weight is always the bias as it is standalone and not responsible for a specific input value.
def predict(row, weights):
	activation = weights[0] # bias
	for i in range(len(row)-1):
		activation += weights[i + 1] * row[i]
	return 1.0 if activation >= 0.0 else 0.0

# Test the predict() function on toy dataset
dataset = [[2.7810836,2.550537003,0],
 [1.465489372,2.362125076,0],
 [3.396561688,4.400293529,0],
 [1.38807019,1.850220317,0],
 [3.06407232,3.005305973,0],
 [7.627531214,2.759262235,1],
 [5.332441248,2.088626775,1],
 [6.922596716,1.77106367,1],
 [8.675418651,-0.242068655,1],
 [7.673756466,3.508563011,1]]
weights = [-0.1, 0.20653640140000007, -0.23418117710000003]
for row in dataset:
    prediction = predict(row, weights)
    print("Expected=%d, Predicted=%d" % (row[-1], prediction))


# Function train_weight() estimates the weight values for our training data using stochastic gradient descent.
# Stochastic gradient descent requires two parameters: 
# Learning Rate: Used to limit the amount each weight is corrected each time it is updated. 
# Epochs: The number of times to run through the training data while updating the weight. 
# There are 3 loops we need to perform in the function: 
# (1) Loop over each epoch. (2) Loop over each row in the training data for an epoch. and (3) Loop over each weight and update it for a row in an epoch. 
# Each weight is updated for each row in the training data, each epoch. 
# Weights are updated based on the error the model made. 
# The error is calculated as the difference between the expected output value and the prediction made with the candidate weights. 
def train_weights(train, l_rate, n_epoch):
	weights = [0.0 for i in range(len(train[0]))]
	for epoch in range(n_epoch):
		sum_error = 0.0
		for row in train:
			prediction = predict(row, weights)
			error = row[-1] - prediction
			sum_error += error**2
			weights[0] = weights[0] + l_rate * error  # Update of bias
			for i in range(len(row)-1):
				weights[i + 1] = weights[i + 1] + l_rate * error * row[i]
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
	return weights 

# Test the train_weights() function
l_rate = 0.1
n_epoch = 5
weights = train_weights(dataset, l_rate, n_epoch)
print(weights)
      
# Next we train a Perceptron model using stochastic gradient descent on the Sonar dataset. 
# Here we assumes that a CSV copy of the dataset is in the current working directory with the file name sonar.all-data.csv. 
# The dataset is first loaded, the string values converted to numeric and the output column is converted from strings to the integer values of 0 to 1.
# This is achieved with helper functions load_csv(), str_column_to_float() and str_column_to_int() to load and prepare the dataset. 
# k-fold cross validation to estimate the performance of the learned model on unseen data. 
# This means that we will construct and evaluate k models and estimate the performance as the mean model error. 
# Classification accuracy will be used to evaluate each model. 
# These behaviors are provided in the cross_validation_split(), accuracy_metric() and evaluate_algorithm() helper functions.
# Perceptron Algorithm on the Sonar Dataset
from random import seed
from random import randrange
from csv import reader

# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)# Make a prediction with weights
	return dataset		
def predict(row, weights):
	activation = weights[0]
	for i in range(len(row)-1):
		activation += weights[i + 1] * row[i]
	return 1.0 if activation >= 0.0 else 0.0

# Estimate Perceptron weights using stochastic gradient descent
def train_weights(train, l_rate, n_epoch):
	weights = [0.0 for i in range(len(train[0]))]
	for epoch in range(n_epoch):
		for row in train:
			prediction = predict(row, weights)
			error = row[-1] - prediction
			weights[0] = weights[0] + l_rate * error
			for i in range(len(row)-1):
				weights[i + 1] = weights[i + 1] + l_rate * error * row[i]
	return weights
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

# Perceptron Algorithm With Stochastic Gradient Descent
def perceptron(train, test, l_rate, n_epoch):
	predictions = list()
	weights = train_weights(train, l_rate, n_epoch)
	for row in test:
		prediction = predict(row, weights)
		predictions.append(prediction)
	return(predictions)

# Test the Perceptron algorithm on the sonar dataset
seed(1)
# load and prepare data
filename = 'sonar.all-data.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert string class to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 3
l_rate = 0.01
n_epoch = 500
scores = evaluate_algorithm(dataset, perceptron, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

      